# reusing the transfer learning re-trained model on new data, back on the same data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import torch
import glob
import os
import pickle
from bs4 import BeautifulSoup
# data_path = '/Users/ruchiraravishankar/Downloads/iam_dataset/'
# data_path = '/content/drive/MyDrive/iam_dataset/'
# save_path = "/content/drive/MyDrive/iam_dataset/saved_model/"

data_path = '/content/drive/MyDrive/testing_proj/iam_dataset/'
save_path = '/content/drive/MyDrive/testing_proj/iam_dataset/saved_model/'

# print(glob.glob(data_path))

In [ ]:
# toggles to change training and nodes

# set model_node to 2,3,4,5,6,7, or 23 for nodes 2 and 3 trained model
model_node = 2

fc_pi = 'fc' 

# # transfer learning toggle
# fc = True
# conv = False
# plain_inf = False

# toggle to save model
save_model = False
# toggle to save results
save_results = True

In [ ]:
labelled_data = np.load(data_path + "labelled_data.npy")

In [ ]:
labelled_data_0 = labelled_data[labelled_data[:,1] == '000']

print(labelled_data_0.shape)

labelled_data_0 = labelled_data_0[:196] # original set of data to the one that its originally trained on
print(labelled_data_0.shape)

print(np.unique(labelled_data_0[:,1]))

In [ ]:
remove_000 = labelled_data[labelled_data[:,1]!='000']

print(np.unique(remove_000[:,1]))
print(np.unique(remove_000[:,1]).shape)

In [ ]:
y = [remove_000[remove_000[:,1]==k] for k in np.unique(remove_000[:,1])]
print(len(y))

In [ ]:
labelled_data_1 = []
for i in y:
    labelled_data_1.append(i[:4]) # original set of data to the one that its originally trained on
print(len(labelled_data_1))

labelled_data_1 = np.array(labelled_data_1)

print(labelled_data_1.shape)

In [ ]:
labelled_data_1=labelled_data_1.reshape(-1,2)

In [ ]:
print(labelled_data_1.shape)

In [ ]:
print(np.unique(labelled_data_1[:,1]))

In [ ]:
labelled_data_new = np.concatenate((labelled_data_0, labelled_data_1))
np.random.shuffle(labelled_data_new)
print(labelled_data_new.shape) # mix of sentences from writer id 0 and others, labelled

In [ ]:
# split before augmenting to ensure the sentences are being split and not the augmented samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(labelled_data_new[:,0], labelled_data_new[:,1], test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
from sklearn.utils import shuffle
from PIL import Image
import random

CROP_SIZE = 113
NUM_LABELS = 50

def get_augmented_sample(sample, label, sample_ratio):
    # Get current image details
    img = Image.open(sample)
    img_width = img.size[0]
    img_height = img.size[1]

    # Compute resize dimensions such that aspect ratio is maintained
    height_fac = CROP_SIZE / img_height
    size = (int(img_width * height_fac), CROP_SIZE)

    # Resize image 
    new_img = img.resize((size), Image.ANTIALIAS)
    new_img_width = new_img.size[0]
    new_img_height = new_img.size[1]

    # Generate a random number of crops of size 113x113 from the resized image
    x_coord = list(range(0, new_img_width - CROP_SIZE))
    num_crops = int(len(x_coord) * sample_ratio)
    random_x_coord = random.sample(x_coord, num_crops)
    
    # Create augmented images (cropped forms) and map them to a label (writer)
    images = []
    labels = []
    for x in random_x_coord:
        img_crop = new_img.crop((x, 0, x + CROP_SIZE, CROP_SIZE))
        # Transform image to an array of numbers
        images.append(np.asarray(img_crop))
        labels.append(label)

    return images, labels

In [ ]:
def augment(data, labels):
    augmented_sample_list = []
    augmented_label_list = []
    for i in range(len(data)):
        augmented_samples, augmented_labels = get_augmented_sample(data[i], labels[i],0.1)
        augmented_sample_list.append(augmented_samples)
        augmented_label_list.append(augmented_labels)
    return augmented_sample_list, augmented_label_list

In [ ]:
augsamps, auglbls = augment(X_train, y_train) # returns two lists of arrays
ausamps_test, auglbls_test = augment(X_test, y_test)

In [ ]:
import operator
from functools import reduce

# flattening the lists
aug_samps = reduce(operator.add, augsamps)
aug_lbls = reduce(operator.add, auglbls)

aug_samps_test = reduce(operator.add, ausamps_test)
aug_lbls_test = reduce(operator.add, auglbls_test)
print(len(aug_samps))
print(len(aug_lbls))

print(len(aug_samps_test))
print(len(aug_lbls_test))

In [ ]:
# converting lists to arrays
aug_samps_arr = np.array(aug_samps)
aug_lbls_arr = np.array(aug_lbls)
aug_samps_arr_test = np.array(aug_samps_test)
aug_lbls_arr_test = np.array(aug_lbls_test)

print(aug_samps_arr.shape)
print(aug_lbls_arr.shape)
print(aug_samps_arr_test.shape)
print(aug_lbls_arr_test.shape)

In [ ]:
X_train = aug_samps_arr
X_train = X_train.reshape(X_train.shape[0], 1, CROP_SIZE, CROP_SIZE)
X_train = X_train.astype('float16')
# X_data /= 255
X_train /= 255.0
y_train = aug_lbls_arr


X_test = aug_samps_arr_test
X_test = X_test.reshape(X_test.shape[0], 1, CROP_SIZE, CROP_SIZE)
X_test = X_test.astype('float16')
X_test /= 255.0
y_test = aug_lbls_arr_test


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
new = np.unique(y_train)
print(new)

y_train[y_train !='000'] = 1
new = np.unique(y_train)
print(new)

y_train[y_train=='000'] = 0
new = np.unique(y_train)
print(new)

print(y_train.shape)

y_train = y_train.astype('float16')

print(np.unique(y_train))

In [ ]:
new = np.unique(y_test)
print(new)

y_test[y_test !='000'] = 1
new = np.unique(y_test)
print(new)

y_test[y_test=='000'] = 0
new = np.unique(y_test)
print(new)

print(y_test.shape)

y_test = y_test.astype('float16')

print(np.unique(y_test))

In [ ]:
torch.manual_seed(42)
CUDA = torch.cuda.is_available()

device = torch.device('cuda' if CUDA else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}
device

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

if CUDA:
  tensor_x_train = torch.Tensor(X_train).type(torch.cuda.FloatTensor) # transform to torch tensor
  tensor_y_train = torch.Tensor(y_train).type(torch.cuda.FloatTensor).long()
else:
  tensor_x_train = torch.Tensor(X_train).type(torch.FloatTensor) # transform to torch tensor
  tensor_y_train = torch.Tensor(y_train).type(torch.FloatTensor).long()

train_dataset = TensorDataset(tensor_x_train,tensor_y_train) # create your dataset
train_dataloader = DataLoader(train_dataset,batch_size=16, shuffle=True, drop_last=True) # create your dataloader

In [ ]:
if CUDA:
  tensor_x_test = torch.Tensor(X_test).type(torch.cuda.FloatTensor) # transform to torch tensor
  tensor_y_test = torch.Tensor(y_test).type(torch.cuda.FloatTensor).long()
else:
  tensor_x_test = torch.Tensor(X_test).type(torch.FloatTensor) # transform to torch tensor
  tensor_y_test = torch.Tensor(y_test).type(torch.FloatTensor).long()

test_dataset = TensorDataset(tensor_x_test,tensor_y_test) # create your datset
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, drop_last=True) # create your dataloader

In [ ]:
tensor_x_train.shape

In [ ]:
from torch import nn
class Binary_Classifier(nn.Module): 
    def __init__(self):
        super(Binary_Classifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv_drop = nn.Dropout2d()
        self.conv3 = nn.Conv2d(20, 30, kernel_size=3)
        self.fc1 = nn.Linear(4320, 1024)
        self.fc2 = nn.Linear(1024, 2)
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(2)
    def forward(self, x):
        x = self.relu(self.max_pool2d(self.conv1(x)))
        x = self.relu(self.max_pool2d(self.conv_drop(self.conv2(x))))
        x = self.relu(self.max_pool2d(self.conv_drop(self.conv3(x))))
        x = x.view(x.shape[0],-1)
        # print(x.shape)
        x = self.relu(self.fc1(x))
        x = self.conv_drop(x)
        x = self.fc2(x)
        return x

In [ ]:
model=Binary_Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
model_node_2_fc = 'model_tf_3_convs_retrain_last_fc_node2.pt'
model_node_2_conv = 'model_tf_3_convs_retrain_last_conv_node2.pt'

model_node_3_fc = 'model_tf_3_convs_retrain_last_fc_node3.pt'
model_node_3_conv = 'model_tf_3_convs_retrain_last_conv_node3.pt'

model_node_4_fc = 'model_3_convs_node_4_fc.pt'
model_node_4_conv = 'model_3_convs_node_4_conv.pt'

model_node_5_fc = 'model_3_convs_node5_fc.pt'
model_node_5_conv = 'model_3_convs_node5_conv.pt'

model_node_6_fc = '3convs_node6_fc.pt'
model_node_6_conv = '3convs_node6_conv.pt'

model_node_7_fc = '3convs_node7_fc.pt'
model_node_7_conv = '3convs_node7_conv.pt'

model_node2_node3 = 'modelreuse_node2convmodel_retrainedfc_on_node3.pt'

In [ ]:
if model_node == 2:
  model_path = model_node_2_conv


elif model_node == 3:
  model_path = model_node_3_conv


elif model_node == 4:
  model_path = model_node_4_conv


elif model_node == 5:
  model_path = model_node_5_conv


elif model_node == 6:
  model_path = model_node_6_conv

elif model_node == 7:
  model_path = model_node_7_conv


elif model_node == 23:
  model_path = model_node2_node3

else:
  print("set model_node variable at the top of the file")

In [ ]:
# if os.path.exists(save_path+'/model_3_convs_node6_conv.pt'):
try:
    print('Loading checkpoint: %s' % save_path + model_path)
    path = torch.load(save_path + model_path)
    # epoch = path['epoch']
    model.load_state_dict(path['model'])
    optimizer.load_state_dict(path['optimizer'])
except Exception as e:
  print("nothing happened")
  print(e)

In [ ]:
print(model.parameters)

In [ ]:
print(device)

In [ ]:
if fc_pi == 'fc':
  for param in model.parameters():
    param.requires_grad = False

In [ ]:
if fc_pi == 'fc':
  num_ftrs = model.fc2.in_features
  print(num_ftrs)
  model.fc2 = nn.Linear(num_ftrs, 2).to(device)

In [ ]:
train_losses = []
train_loss_record = []

In [ ]:
from datetime import datetime
train_log_string = '%s :: Epoch %i :: Iter %i / %i :: train loss: %0.4f'
def train(epoch):
    print('Start Epoch {} Training...'.format(epoch))
    model.train()
    train_loss = []
    train_loss_sum = []
    for idx, (data, target) in enumerate(train_dataloader):
        optimizer.zero_grad()
        #  forward-pass
        output = model(data)
        loss = criterion(output, target)
        #backward-pass
        loss.backward()
        # Update the parameters
        optimizer.step()
        train_loss.append(loss.data.cpu().numpy())
        train_loss_sum.append(loss)
        if ((idx +1) % 5000) == 0:
          print(train_log_string % (datetime.now(), epoch, idx + 1, len(train_dataloader), np.mean(train_loss)))
          train_loss = []
    mean_tr = torch.mean(torch.stack(train_loss_sum))
    print('Train Loss at epoch {}: {}\n'.format(epoch, mean_tr))
    train_loss_record.append(mean_tr)
    return train_loss_record

In [ ]:
if fc_pi == 'fc':
  for epoch in range(15):
    tlr = train(epoch)

In [ ]:
import matplotlib.pyplot as plt
if fc_pi == 'fc':
  temp = [t.detach().cpu().numpy() for t in tlr]
  arr = np.array(temp)
  print(arr.shape)

  plt.plot(arr)

In [ ]:
result = []
prediction = []
model.eval()
with torch.no_grad():
    for idx, (data, target) in enumerate(test_dataloader):
        output = model(data)
        output_sm = nn.Softmax(dim=1)(output)
        _, predicted = torch.max(output, 1)
        prediction.append(predicted)
        result.append(output_sm)
        
result = torch.stack(result).detach().cpu().numpy()
prediction = torch.stack(prediction).detach().cpu().numpy()

print(result.shape, prediction.shape)

In [ ]:
print(prediction.reshape(-1).shape)

In [ ]:
import pandas as pd
res = pd.DataFrame(result.reshape(-1,2))
pred = pd.DataFrame(prediction.reshape(-1))

# res.to_csv("/content/drive/MyDrive/iam_dataset/saved_model/results_tf_reuse_original_data_conv.csv", header=False, index=False)
# pred.to_csv("/content/drive/MyDrive/iam_dataset/saved_model/pred_tf_reuse_original_data_conv.csv", header=False, index=False)

In [ ]:
print(res.shape, pred.shape)

In [ ]:
y_test_new = y_test[:result.reshape(-1,2).shape[0]]

In [ ]:
print(y_test_new.shape)

In [ ]:
result_df = pd.DataFrame({'ground truth': y_test_new, 'NN output': prediction.reshape(-1), 'probs 0': res[0], 'probs 1': res[1]})
result_df

In [ ]:
# work out the accuracy and other metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
actual = result_df['ground truth']
predicted = result_df['NN output']
# confusion matrix
matrix = confusion_matrix(actual, predicted, labels=[0,1])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(actual, predicted, labels=[0,1]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

# classification report for precision, recall f1-score and accuracy
matrix_2 = classification_report(actual,predicted,labels=[0,1])
print('Classification report : \n',matrix_2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
df_cm = pd.DataFrame(matrix, index = [i for i in "01"],
                  columns = [i for i in "01"])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
matrix_3 = classification_report(actual,predicted,labels=[0,1], output_dict=True)
print('Classification report : \n',matrix_3)
accuracy_score(actual, predicted)

In [ ]:
matrix_df = pd.DataFrame(matrix).transpose()
matrix_3_df = pd.DataFrame(matrix_3).transpose()
# matrix_df.to_csv("/content/drive/MyDrive/iam_dataset/saved_model/conf_matrix_transf_learn_conv_newdata_2.csv", header=False, index=False)

if save_results:
  if model_node == 2:
    if fc_pi == 'fc':
      matrix_3_df.to_csv(save_path + "classification_report_transf_learn_reuse_node2_conv_fc_retrain.csv", header=True, index=True)

    elif fc_pi == 'pi':
      matrix_3_df.to_csv(save_path + "classification_report_transf_learn_reuse_node2_conv_pi.csv", header=True, index=True)

    else:
      print("set model_node or fc_pi")
    
  elif model_node == 3:
    if fc_pi == 'fc':
      matrix_3_df.to_csv(save_path + "classification_report_transf_learn_reuse_node3_conv_fc_retrain.csv", header=True, index=True)

    elif fc_pi == 'pi':
      matrix_3_df.to_csv(save_path + "classification_report_transf_learn_reuse_node3_conv_pi.csv", header=True, index=True)

    else:
      print("set model_node or fc_pi")

  elif model_node == 4:
    if fc_pi == 'fc':
      matrix_3_df.to_csv(save_path + "classification_report_modelreuse_node4conv_retrainfc_on_node1.csv", header=True, index=True)

    elif fc_pi == 'pi':
      matrix_3_df.to_csv(save_path + "classification_report_modelreuse_node4conv_pi_on_node1.csv", header=True, index=True)

    else:
      print("set model_node or fc_pi")
  
  elif model_node == 5:
    if fc_pi == 'fc':
      matrix_3_df.to_csv(save_path + "classification_report_modelreuse_node5conv_retrainfc_on_node1.csv", header=True, index=True)

    elif fc_pi == 'pi':
      matrix_3_df.to_csv(save_path + "classification_report_modelreuse_node5conv_pi_on_node1.csv", header=True, index=True)

    else:
      print("set model_node or fc_pi")
  
  elif model_node == 6:
    if fc_pi == 'fc':
      matrix_3_df.to_csv(save_path + "classification_report_modelreuse_node6conv_retrainfc_on_node1.csv", header=True, index=True)

    elif fc_pi == 'pi':
      matrix_3_df.to_csv(save_path + "classification_report_modelreuse_node6conv_pi_on_node1.csv", header=True, index=True)

    else:
      print("set model_node or fc_pi")
  
  elif model_node == 7:
    if fc_pi == 'fc':
      matrix_3_df.to_csv(save_path + "classification_report_modelreuse_node7conv_retrainfc_on_node1.csv", header=True, index=True)

    elif fc_pi == 'pi':
      matrix_3_df.to_csv(save_path + "classification_report_modelreuse_node7conv_pi_on_node1.csv", header=True, index=True)

    else:
      print("set model_node or fc_pi")  

  elif model_node == 23:
      matrix_3_df.to_csv(save_path + "classification_report_modelreuse_node2and3model_on_node1.csv", header=True, index=True)
else:
    print("set model_node")

# pd.to_csv("/content/drive/MyDrive/iam_dataset/saved_model/classification_report.txt",matrix)
